In [17]:
import gc
import numpy as np
import pandas as pd
import sklearn.ensemble
import xgboost as xgb

In [2]:
trainfiles = ['train_categorical.csv',
                  'train_date.csv',
                  'train_numeric.csv']
testfiles = ['test_categorical.csv',
                 'test_date.csv',
                 'test_numeric.csv']

cols = [['Id',
             'L1_S24_F1559', 'L3_S32_F3851',
             'L1_S24_F1827', 'L1_S24_F1582',
             'L3_S32_F3854', 'L1_S24_F1510',
             'L1_S24_F1525'],
            ['Id',
             'L3_S30_D3496', 'L3_S30_D3506',
             'L3_S30_D3501', 'L3_S30_D3516',
             'L3_S30_D3511'],
            ['Id',
             'L1_S24_F1846', 'L3_S32_F3850',
             'L1_S24_F1695', 'L1_S24_F1632',
             'L3_S33_F3855', 'L1_S24_F1604',
             'L3_S29_F3407', 'L3_S33_F3865',
             'L3_S38_F3952', 'L1_S24_F1723',
             'Response']]
cat_features = 94

In [3]:
traindata = None
for i, f in enumerate(trainfiles):
    print(f)
    subset = None
    for i, chunk in enumerate(pd.read_csv(f, usecols=cols[i], chunksize=50000, low_memory=False)):
        print(i)
        if subset is None:
            subset = chunk.copy()
        else:
            subset = pd.concat([subset, chunk])
        del chunk
        gc.collect()
    if traindata is None:
        traindata = subset.copy()
    else:
        traindata = pd.merge(traindata, subset.copy(), on="Id")
    del subset
    gc.collect()

train_categorical.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
train_date.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
train_numeric.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [4]:
testdata = None
del cols[2][-1]
for i, f in enumerate(testfiles):
    print(f)
    subset = None
    for i, chunk in enumerate(pd.read_csv(f, usecols=cols[i], chunksize=25000, low_memory=False)):
        print(i)
        if subset is None:
            subset = chunk.copy()
        else:
            subset = pd.concat([subset, chunk])
        del chunk
        gc.collect()
    if testdata is None:
        testdata = subset.copy()
    else:
        testdata = pd.merge(testdata, subset.copy(), on="Id")
    del subset
    gc.collect()

test_categorical.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
test_date.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
test_numeric.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [5]:
def LeaveOneOut(data1, data2, columnName, useLOO=False):
    grpOutcomes = data1.groupby(columnName)['Response'].mean().reset_index()
    grpCount = data1.groupby(columnName)['Response'].count().reset_index()
    grpOutcomes['cnt'] = grpCount.Response
    if(useLOO):
        grpOutcomes = grpOutcomes[grpOutcomes.cnt > 1]
    grpOutcomes.drop('cnt', inplace=True, axis=1)
    outcomes = data2['Response'].values
    x = pd.merge(data2[[columnName, 'Response']], grpOutcomes,
                 suffixes=('x_', ''),
                 how='left',
                 on=columnName,
                 left_index=True)['Response']
    if(useLOO):
        x = ((x*x.shape[0])-outcomes)/(x.shape[0]-1)
        #  x = x + np.random.normal(0, .01, x.shape[0])
    return x.fillna(x.mean())


In [6]:
testdata['Response'] = 0
visibletraindata = traindata[::2]
blindtraindata = traindata[1::2]
print(blindtraindata.columns)
for i in range(2):
    for col in cols[i][1:]:
        print(col)
        blindtraindata.loc[:, col] = LeaveOneOut(visibletraindata,
                                                 blindtraindata,
                                                 col, False).values
        testdata.loc[:, col] = LeaveOneOut(visibletraindata,
                                           testdata, col, False).values
del visibletraindata

testdata.drop('Response', inplace=True, axis=1)
traindata = blindtraindata
gc.collect()

Index(['Id', 'L1_S24_F1510', 'L1_S24_F1525', 'L1_S24_F1559', 'L1_S24_F1582',
       'L1_S24_F1827', 'L3_S32_F3851', 'L3_S32_F3854', 'L3_S30_D3496',
       'L3_S30_D3501', 'L3_S30_D3506', 'L3_S30_D3511', 'L3_S30_D3516',
       'L1_S24_F1604', 'L1_S24_F1632', 'L1_S24_F1695', 'L1_S24_F1723',
       'L1_S24_F1846', 'L3_S29_F3407', 'L3_S32_F3850', 'L3_S33_F3855',
       'L3_S33_F3865', 'L3_S38_F3952', 'Response'],
      dtype='object')
L1_S24_F1559


/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


L3_S32_F3851
L1_S24_F1827
L1_S24_F1582
L3_S32_F3854
L1_S24_F1510
L1_S24_F1525
L3_S30_D3496
L3_S30_D3506
L3_S30_D3501
L3_S30_D3516
L3_S30_D3511


77

In [8]:
traindata.fillna(0, inplace=True)
testdata.fillna(0, inplace=True)

In [24]:
clf = sklearn.ensemble.GradientBoostingClassifier(n_estimators=10000, max_depth=5, verbose=1)

In [25]:

clf.fit(traindata.values[:, 1:-1], traindata.values[:,-1])

      Iter       Train Loss   Remaining Time 
         1           0.0654          123.65m
         2           0.0646          133.05m
         3           0.0643          137.07m
         4           0.1072          138.23m
         5           0.1068          137.94m
         6           0.1066          138.74m
         7           0.1064          136.21m
         8           0.1062          137.65m
         9           0.1059          136.98m
        10           0.1058          135.93m
        20           0.0927          126.20m
        30           0.0923          120.02m
        40           0.0920          116.84m
        50           0.0918          116.61m
        60           0.0917          114.96m
        70           0.0916          115.10m
        80           0.0915          115.26m
        90           0.0914          114.44m
       100           0.0913          114.83m
       200           0.0904          111.13m
       300           0.0896          107.83m
       40

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=5, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10000,
              presort='auto', random_state=None, subsample=1.0, verbose=1,
              warm_start=False)

In [29]:
# num_rounds = 50

# params = {}
# params['objective'] = "binary:logistic"
# params['eta'] = 0.021
# params['max_depth'] = 7
# params['colsample_bytree'] = 0.82
# params['min_child_weight'] = 3
# params['base_score'] = 0.005
# params['silent'] = False

# features = traindata.columns[1:-1]

# dtrain = xgb.DMatrix(traindata[features],
#                     train.Response,
#                     silent=True)
# dtest = xgb.DMatrix(testdata[features],
#                     silent=True)

# watchlist = [(dtrain, 'train'), (dtrain, 'val')]
# clf = xgb.train(params, dtrain,
#                         num_boost_round=num_rounds,
#                         evals=watchlist,
#                         early_stopping_rounds=20,
#                         maximize=True
#                         )

In [28]:
ans = clf.predict(testdata.values[:, 1:])

In [32]:
ans = ans.astype(int)

In [33]:
submission = pd.read_csv("sample_submission.csv")
submission.iloc[:, 1] = ans
submission.to_csv('answer.gb.csv', index=None)
# 0.20332